In [1]:
import helpers.utilities
%run helpers/notebook_setup.ipynb

In [2]:
from helpers.gse import collection_to_R, nice_kegg, formatter_to_fix_letter_case
from gsea_api.molecular_signatures_db import MolecularSignaturesDatabase, GeneMatrixTransposed

Pathway enrichment analyses of results for specific analysis methods are presented in respective notebooks.

This notebook presents the methods and data sources for these analyses:

In [3]:
from jupyter_helpers.selective_import import notebooks_importer

from analyses.integration.Uniform_preprocessing import raw_protein_matrix, raw_rna_matrix

Importing Jupyter notebook from /home/krassowski/meningitis-integration/analyses/integration/Uniform_preprocessing.ipynb


In [4]:
db = MolecularSignaturesDatabase('data/msigdb', version='7.1')
reactome_new_symbol = GeneMatrixTransposed.from_gmt('data/ReactomePathways.gmt', name='reactome_new_symbol')
kegg_pathways_symbol = db.load(gene_sets='c2.cp.kegg', id_type='symbols').format_names(nice_kegg)
reactome_pathways_symbol = db.load(gene_sets='c2.cp.reactome', id_type='symbols').format_names(
    formatter_to_fix_letter_case(reactome_new_symbol)
)
gene_ontology_symbol = db.load(gene_sets='c5.all', id_type='symbols')
immune_signature_symbol = db.load(gene_sets='c7.all', id_type='symbols')
hallmarks_symbol = db.load(gene_sets='h.all', id_type='symbols').format_names(
    lambda x: x.replace('HALLMARK_', '').replace('_', ' ')
)

symbols = {*raw_rna_matrix.index, *raw_protein_matrix.index}

collections = {
    'hallmarks': [hallmarks_symbol, symbols],
    'hallmarks_for_rna': [hallmarks_symbol, set(raw_rna_matrix.index)],
    'hallmarks_for_protein': [hallmarks_symbol, set(raw_protein_matrix.index)],

    'reactome': [reactome_pathways_symbol, symbols],
    'reactome_for_rna': [reactome_pathways_symbol, set(raw_rna_matrix.index)],
    'reactome_for_protein': [reactome_pathways_symbol, set(raw_protein_matrix.index)]
}


def export_collections_to_r(collections):
    for name, (collection, genes) in collections.items():
        after_trimming = collection_to_R(collection, trim_to=genes, name=name)
        print(name, len(after_trimming))


export_collections_to_r(collections)

/home/krassowski/.pyenv/versions/3.7.6/envs/meningitis/lib/python3.7/site-packages/gsea_api/molecular_signatures_db.py:65: UserWarning: Provided gene sets are not redundant; there are 117 gene sets having more than one name assigned; use `find_redundant()` to investigate further.
  warn(message)
/home/krassowski/.pyenv/versions/3.7.6/envs/meningitis/lib/python3.7/site-packages/gsea_api/molecular_signatures_db.py:65: UserWarning: Provided gene sets are not redundant; there are 40 gene sets having more than one name assigned; use `find_redundant()` to investigate further.
  warn(message)


hallmarks 50
hallmarks_for_rna 50
hallmarks_for_protein 49
reactome 1431
reactome_for_rna 1430


/home/krassowski/.pyenv/versions/3.7.6/envs/meningitis/lib/python3.7/site-packages/gsea_api/molecular_signatures_db.py:65: UserWarning: Provided gene sets are not redundant; following gene sets are identical: 'Telomere Maintenance' and 'Extension of Telomeres' (44 genes)
  warn(message)


reactome_for_protein 581


/home/krassowski/.pyenv/versions/3.7.6/envs/meningitis/lib/python3.7/site-packages/gsea_api/molecular_signatures_db.py:65: UserWarning: Provided gene sets are not redundant; there are 104 gene sets having more than one name assigned; use `find_redundant()` to investigate further.
  warn(message)
/home/krassowski/.pyenv/versions/3.7.6/envs/meningitis/lib/python3.7/site-packages/gsea_api/molecular_signatures_db.py:75: UserWarning: There are 281 empty gene sets: use `empty_gene_sets` property to investigate further.
  warn(f'There are {len(empty_gene_sets)} empty gene sets: {empty_message}')
/home/krassowski/.pyenv/versions/3.7.6/envs/meningitis/lib/python3.7/site-packages/gsea_api/molecular_signatures_db.py:79: UserWarning: 281 empty gene sets were removed.
  warn(f'{len(empty_gene_sets)} empty gene sets were removed.')
